In [1]:
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 6.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from datasets import load_dataset, interleave_datasets, concatenate_datasets

In [14]:
table_names = [
              'abstract_algebra',
               'anatomy',
               'astronomy',
               'business_ethics',
               'clinical_knowledge',
               'college_biology',
               'college_chemistry',
               'college_computer_science',
               'college_mathematics',
               'college_medicine',
               'college_physics',
               'computer_security',
               'conceptual_physics',
               'econometrics',
               'electrical_engineering',
               'elementary_mathematics',
               'formal_logic',
               'global_facts',
               'high_school_biology',
               'high_school_chemistry',
               'high_school_computer_science',
               'high_school_european_history',
               'high_school_geography',
               'high_school_government_and_politics',
               'high_school_macroeconomics',
               'high_school_mathematics',
               'high_school_microeconomics',
               'high_school_physics',
               'high_school_psychology',
               'high_school_statistics',
               'high_school_us_history',
               'high_school_world_history',
               'human_aging',
               'human_sexuality',
               'international_law',
               'jurisprudence',
               'logical_fallacies',
               'machine_learning',
               'management',
               'marketing',
               'medical_genetics',
               'miscellaneous',
               'moral_disputes',
               'moral_scenarios',
               'nutrition',
               'philosophy',
               'prehistory',
               'professional_accounting',
               'professional_law',
               'professional_medicine',
               'professional_psychology',
               'public_relations',
               'security_studies',
               'sociology',
               'us_foreign_policy',
               'virology',
               'world_religions']
options = ['A', 'B', 'C', 'D']

In [15]:
options = ["A", "B", "C", "D"]
# options = ["a", "b", "c", "d"]

def map_to_desired_position(example, id, new_answer_idx, option_prefix_char, option_suffix_char):
  answer_idx = example["answer"]

  modified_choices = example["choices"].copy()
  temp = modified_choices[answer_idx]
  modified_choices[answer_idx] = modified_choices[new_answer_idx]
  modified_choices[new_answer_idx] = temp

  modified_options = options.copy()
  temp = modified_options[answer_idx]
  modified_options[answer_idx] = modified_options[new_answer_idx]
  modified_options[new_answer_idx] = temp
  num_options = len(example["choices"])

  options_str = ''
  for option in modified_options:
    options_str += option
  # print(options_str)

  question_text = "Question: " + example["question"] + "\nOptions: "
  for option_idx in range(0, num_options):
    question_text += "\n" + option_prefix_char + modified_options[option_idx] + option_suffix_char + " " + modified_choices[option_idx]
  question_text += "\nAnswer: \n"
  return {
      "question_id": id,
      "question": question_text,
      "gold_answer": options[answer_idx],
      "gold_position": new_answer_idx,
      "modified_options": options_str
  }

In [16]:
def create_dataset_by_varying_positions(dataset, table_name, folder_path, option_prefix_char, option_suffix_char):
  dataset_1 = dataset.map(map_to_desired_position, with_indices=True, fn_kwargs={
      "new_answer_idx": 0,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
  })
  dataset_2 = dataset.map(map_to_desired_position, with_indices=True, fn_kwargs={
      "new_answer_idx": 1,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
  })
  dataset_3 = dataset.map(map_to_desired_position, with_indices=True, fn_kwargs={
      "new_answer_idx": 2,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
  })
  dataset_4 = dataset.map(map_to_desired_position, with_indices=True, fn_kwargs={
      "new_answer_idx": 3,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
  })

  datasets = [dataset_1, dataset_2, dataset_3, dataset_4]

  merged_dataset = interleave_datasets(datasets)
  merged_dataset = merged_dataset.shuffle()

  path = folder_path + table_name + ".csv"
  merged_dataset.to_csv(path)

# create_dataset_by_varying_positions("abstract_algebra", "/content/drive/MyDrive/685 Project/mmlu 01/varying_position/")

In [17]:
idx_to_option_mapping = {0: "A", 1: "B", 2: "C", 3: "D"}
# idx_to_option_mapping = {0: "a", 1: "b", 2: "c", 3: "d"}

def map_to_desired_option(example, id, new_answer_idx, option_prefix_char, option_suffix_char):
  answer_idx = example["answer"]
  choices = example["choices"]

  modified_options = options.copy()
  temp = modified_options[answer_idx]
  modified_options[answer_idx] = modified_options[new_answer_idx]
  modified_options[new_answer_idx] = temp
  num_options = len(example["choices"])

  options_str = ''
  for option in modified_options:
    options_str += option

  question_text = "Question: " + example["question"] + "\nOptions: "
  for option_idx in range(0, num_options):
    question_text += "\n" + option_prefix_char + modified_options[option_idx] + option_suffix_char + " " + choices[option_idx]
  question_text += "\nAnswer: \n"
  return {
      "question_id": id,
      "question": question_text,
      "gold_answer": idx_to_option_mapping[new_answer_idx],
      "modified_options": options_str
  }

In [18]:
def create_dataset_by_varying_options(dataset, table_name, folder_path, option_prefix_char, option_suffix_char):
  dataset_1 = dataset.map(map_to_desired_option, with_indices=True, fn_kwargs={
      "new_answer_idx": 0,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
  })
  dataset_2 = dataset.map(map_to_desired_option, with_indices=True, fn_kwargs={
      "new_answer_idx": 1,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
  })
  dataset_3 = dataset.map(map_to_desired_option, with_indices=True, fn_kwargs={
      "new_answer_idx": 2,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
  })
  dataset_4 = dataset.map(map_to_desired_option, with_indices=True, fn_kwargs={
      "new_answer_idx": 3,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
  })

  datasets = [dataset_1, dataset_2, dataset_3, dataset_4]

  merged_dataset = interleave_datasets(datasets)
  merged_dataset = merged_dataset.shuffle()

  path = folder_path + table_name + ".csv"
  merged_dataset.to_csv(path)

In [19]:
def pick_samples(dataset, n):
  # dataset = load_dataset("cais/mmlu", table_name)["test"]
  dataset_0 = dataset.filter(lambda example: example["answer"] == 0)
  dataset_1 = dataset.filter(lambda example: example["answer"] == 1)
  dataset_2 = dataset.filter(lambda example: example["answer"] == 2)
  dataset_3 = dataset.filter(lambda example: example["answer"] == 3)
  # n = min(len(dataset_0), len(dataset_1), len(dataset_2), len(dataset_3))
  # print(f"n: {n}")
  # print(f"{len(dataset_0), len(dataset_1), len(dataset_2), len(dataset_3)}")

  try:
    dataset_0 = dataset_0.select(range(n))
    dataset_1 = dataset_1.select(range(n))
    dataset_2 = dataset_2.select(range(n))
    dataset_3 = dataset_3.select(range(n))
  except:
    print("not enough samples")

  target = interleave_datasets([dataset_0, dataset_1, dataset_2, dataset_3])
  target = target.shuffle()
  return target

In [20]:
def create_datasets(test_datasets, table_names, folder_path, option_prefix_char = '', option_suffix_char = '.'):
  num = len(test_datasets)
  for idx in range(num):
    dataset = pick_samples(test_datasets[idx], n=25)
    create_dataset_by_varying_positions(dataset, table_names[idx], folder_path + "varying_position/", option_prefix_char, option_suffix_char)
    create_dataset_by_varying_options(dataset, table_names[idx], folder_path + "varying_option/", option_prefix_char, option_suffix_char)

In [21]:
train_val_test_tables = [
              # 'abstract_algebra',
               'anatomy',
               'astronomy',
               'business_ethics',
               'clinical_knowledge',
               'college_biology',
               'college_chemistry',
               'college_computer_science',
               'college_mathematics',
              #  'college_medicine',
               'college_physics',
               'computer_security',
              #  'conceptual_physics',
               'econometrics',
               'electrical_engineering',
               'elementary_mathematics',
               'formal_logic',
               'global_facts',
               'high_school_biology',
               'high_school_chemistry',
               'high_school_computer_science',
               'high_school_european_history',
               'high_school_geography',
               'high_school_government_and_politics',
               'high_school_macroeconomics',
              #  'high_school_mathematics',
               'high_school_microeconomics',
               'high_school_physics',
               'high_school_psychology',
               'high_school_statistics',
               'high_school_us_history',
               'high_school_world_history',
               'human_aging',
               'human_sexuality',
               'international_law',
               'jurisprudence',
               'logical_fallacies',
               'machine_learning',
               'management',
               'marketing',
               'medical_genetics',
               'miscellaneous',
               'moral_disputes',
               'moral_scenarios',
               'nutrition',
              #  'philosophy',
              #  'prehistory',
               'professional_accounting',
              #  'professional_law',
               'professional_medicine',
               'professional_psychology',
               'public_relations',
               'security_studies',
               'sociology',
               'us_foreign_policy',
               'virology',
               'world_religions'
              ]
options = ['A', 'B', 'C', 'D']

In [22]:
def preprocess_question(example, id):
  num_options = len(options)
  choices = example["choices"]

  question_text = "Question: " + example["question"] + "\nOptions: "
  for option_idx in range(0, num_options):
    question_text += "\n" + options[option_idx] + ". " + choices[option_idx]
  question_text += "\nAnswer: \n"
  return {
      "question_id": id,
      "question": question_text
  }

In [ ]:
def create_train_dev_test_datasets(tables, folder_path):
  l_train_sets = []
  l_val_sets = []
  l_test_sets = []

  for table_name in tables:
    dataset = load_dataset("cais/mmlu", table_name)["test"]
    split1 = dataset.train_test_split(test_size=0.2)
    split2 = dataset.train_test_split(test_size=0.1)
    train_set = split2["train"]
    val_set = split2["test"]
    test_set = split1["test"]
    print(f"{train_set}, {val_set}, {test_set}")
    l_train_sets.append(train_set)
    l_val_sets.append(val_set)
    l_test_sets.append(test_set)

  train_dataset = concatenate_datasets(l_train_sets)
  train_dataset = train_dataset.map(preprocess_question, with_indices=True)
  train_dataset = train_dataset.shuffle()
  train_dataset.to_csv(f"{folder_path}trainset.csv")

  val_dataset = concatenate_datasets(l_val_sets).map(preprocess_question, with_indices=True).shuffle()
  val_dataset.to_csv(f"{folder_path}valset.csv")

  test_dataset = concatenate_datasets(l_test_sets)
  create_datasets([test_dataset], ["testset"], folder_path)

  return train_dataset, val_dataset, l_test_sets

folder_path = "/content/drive/MyDrive/685 Project/mmlu 01/"
create_train_dev_test_datasets(train_val_test_tables, folder_path)

NameError: name 'train_val_test_tables' is not defined

In [23]:
# Appends gold_answer to train and validation set
def append_gold_answer(example):
  answer = example["answer"]

  return {'gold_answer': options[answer]}


folder_path = "/content/drive/MyDrive/685 Project/mmlu 01/"

# Add a column "gold_answer" to train and val sets
for input_path in [f'{folder_path}trainset.csv', f'{folder_path}valset.csv']:
  dataset = load_dataset('csv', data_files = input_path, split="train")
  dataset = dataset.map(append_gold_answer)
  dataset.to_csv(input_path)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/9963 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1132 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
test_table_names = [
          'abstract_algebra',
          'college_medicine',
          'conceptual_physics',
          'high_school_mathematics',
          'philosophy',
          'prehistory',
          'professional_law'
]

# Creating OOD Testset tables
def test_tables(table_names, folder_path, option_prefix_char="", option_suffix_char="."):
  test_datasets = []
  for table in table_names:
    test_datasets.append(load_dataset("cais/mmlu", table)["test"])
  create_datasets(test_datasets, table_names, folder_path, option_prefix_char, option_suffix_char)

In [ ]:
# Default format A. B. C. D.
folder_path = "/content/drive/MyDrive/685 Project/mmlu 01/"
test_tables(test_table_names, folder_path)

In [ ]:
# Creating data changing option formats (for ablation studies)
# Format 1 A) B) C) D)
option_prefix_char = ''
option_suffix_char = ')'
folder_path = "/content/drive/MyDrive/685 Project/mmlu 01/ablation_data/ablation1/"
test_tables(test_table_names, folder_path, option_prefix_char, option_suffix_char)

In [ ]:
# Format 2 (A) (B) (C) (D)
option_prefix_char = '('
option_suffix_char = ')'
folder_path = "/content/drive/MyDrive/685 Project/mmlu 01/ablation_data/ablation2/"
test_tables(test_table_names, folder_path, option_prefix_char, option_suffix_char)

In [ ]:
# Create testset.csv for ablation studies for format 1, 2.
options = ['A', 'B', 'C', 'D']

def change_option_format(example, prefix_option_char, suffix_option_char):
  question = example["question"]
  for option in options:
    question = question.replace(f"{option}. ", f"{prefix_option_char}{option}{suffix_option_char} ")
  return {"question": question}

def create_ablation_dataset(input_path, output_path, prefix_option_char, suffix_option_char):
  dataset = load_dataset('csv', data_files = input_path)
  dataset = dataset.map(change_option_format, fn_kwargs={"prefix_option_char": '', "suffix_option_char": ')'})
  dataset["train"].to_csv(output_path)

def create_ablation_datasets(ablation_folder, prefix_option_char, suffix_option_char):
  main_folder = "/content/drive/MyDrive/685 Project/mmlu 01/"
  for is_varying_option in [True, False]:
    sub_folder = "varying_option" if is_varying_option else "varying_position"
    input_data = f"{main_folder}{sub_folder}/testset.csv"
    output_data = f"{main_folder}ablation_data/{ablation_folder}/{sub_folder}/testset.csv"
    create_ablation_dataset(input_data, output_data, prefix_option_char, suffix_option_char)

create_ablation_datasets("ablation1", '', ')')
create_ablation_datasets("ablation2", '(', ')')

/content/drive/MyDrive/685 Project/mmlu 01/varying_position/testset.csv -> /content/drive/MyDrive/685 Project/mmlu 01/ablation_data/ablation2/varying_position/testset.csv


Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
# Create testset.csv for ablation studies for format 3.
options = ['A', 'B', 'C', 'D']
modified_options = ['a', 'b', 'c', 'd']

def change_option_format(example):
  question = example["question"]
  for idx, option in enumerate(options):
    question = question.replace(f"{option}. ", f"{modified_options[idx]}. ")
  return {"question": question}

def create_ablation_dataset(input_path, output_path):
  dataset = load_dataset('csv', data_files = input_path)
  dataset = dataset.map(change_option_format)
  dataset["train"].to_csv(output_path)

main_folder = "/content/drive/MyDrive/685 Project/mmlu 01/"
input_data = f"{main_folder}varying_option/testset.csv"
output_data = f"{main_folder}ablation_data/ablation3/varying_option/testset.csv"
create_ablation_dataset(input_data, output_data)

Map:   0%|          | 0/7904 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

In [ ]:
# Not WORKING!
# # Create testset.csv for ablation studies for format 1, 2.
# options = ['A', 'B', 'C', 'D']

# def change_option_format(example, prefix_option_char, suffix_option_char):
#   question = example["question"]
#   for option in options:
#     question = question.replace(f"{option}. ", f"{prefix_option_char}{option}{suffix_option_char} ")
#   return {"question": question}

# def create_permuted_dataset(input_path, output_path):
#   dataset = load_dataset('csv', data_files = input_path)
#   dataset = dataset.map(change_option_format, fn_kwargs={"prefix_option_char": '', "suffix_option_char": ')'})
#   dataset["train"].to_csv(output_path)

# main_folder = "/content/drive/MyDrive/685 Project/mmlu 01/"
# input_data = f"{main_folder}trainset.csv"
# output_data = f"{main_folder}permuted_trainset.csv"
# create_permuted_dataset(input_data, output_data)

In [ ]:
# Creating a balanced trainset
main_folder = "/content/drive/MyDrive/685 Project/mmlu 01/"
trainset = load_dataset('csv', data_files = f"{main_folder}trainset.csv", split="train")
trainset = pick_samples(trainset, n=2200)
trainset.to_csv(f"{main_folder}balanced_trainset.csv")

Creating CSV from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

5345898

In [24]:
def map_to_desired_position2(example, id, new_answer_idx, option_prefix_char='', option_suffix_char='.'):
  answer_idx = example["answer"]

  modified_choices = example["choices"].copy()
  temp = modified_choices[answer_idx]
  modified_choices[answer_idx] = modified_choices[new_answer_idx]
  modified_choices[new_answer_idx] = temp

  return {
      "choices": modified_choices,
      "answer": new_answer_idx,
  }

def append_gold_answer(example):
  answer = example["answer"]

  return {'gold_answer': options[answer]}

In [25]:
def create_permuted_trainset(dataset, option_prefix_char='', option_suffix_char='.'):
  tables = []

  for position in [0, 1, 2, 3]:
    dataset_i = dataset.map(map_to_desired_position2, with_indices=True, fn_kwargs={
      "new_answer_idx": position,
      "option_prefix_char": option_prefix_char,
      "option_suffix_char": option_suffix_char
    })
    for option in [0, 1, 2, 3]:
      dataset_ij = dataset_i.map(map_to_desired_option, with_indices=True, fn_kwargs={
        "new_answer_idx": option,
        "option_prefix_char": option_prefix_char,
        "option_suffix_char": option_suffix_char
      })
      tables.append(dataset_ij)

  permuted_dataset = concatenate_datasets(tables).shuffle()
  return permuted_dataset

In [26]:
# For 8K
permuted_train_table_names_8K = [
  'professional_psychology',
  'electrical_engineering',
  'college_biology',
  'marketing',
  'high_school_world_history'
]

# For 16K
permuted_train_table_names_16K = [
  'high_school_microeconomics',
  'high_school_biology',
  'high_school_world_history',
  'high_school_government_and_politics',
  'high_school_psychology',
  'high_school_chemistry',
  'moral_disputes',
  'high_school_geography',
  'marketing',
  'sociology'
]

# For 32 K
permuted_train_table_names_32K = [
  'professional_psychology',
  'electrical_engineering',
  'college_biology',
  'marketing',
  'high_school_microeconomics',
  'high_school_biology',
  'high_school_world_history',
  'high_school_government_and_politics',
  'high_school_psychology',
  'high_school_chemistry',
  'moral_disputes',
  'high_school_geography',
  'sociology',
  'human_aging',
  'security_studies',
  'logical_fallacies',
  'professional_accounting',
  'miscellaneous',
  'astronomy',
  'world_religions',
]

In [27]:
# Create trainset with the same question permuted with different options and different positions, that is 16 permutations per set.
def create_permuted_set(table_names, mmlu_01, file_name):
  train_tables = []
  val_tables = []
  test_tables = []

  for table_name in table_names:
    print(table_name)
    mmlu = load_dataset("cais/mmlu", table_name)["test"]
    # split1 = mmlu.train_test_split(test_size=0.2)
    # split2 = mmlu.train_test_split(test_size=0.1)
    # train_set = split2["train"].select(range(100))
    # val_set = split2["test"]
    # test_set = split1["test"]
    train_set = mmlu.select(range(100))
    rest_set = mmlu.select(range(100, len(mmlu)))
    split = rest_set.train_test_split(test_size=0.4)
    val_set = split["train"]
    test_set = split["test"]

    train_tables.append(train_set)
    val_tables.append(val_set)
    test_tables.append(test_set)

  train_dataset = concatenate_datasets(train_tables)
  train_dataset = create_permuted_trainset(train_dataset)
  train_dataset.to_csv(f"{mmlu_01}permuted_trainset_{file_name}.csv")

  val_dataset = concatenate_datasets(val_tables).map(preprocess_question, with_indices=True)
  val_dataset = val_dataset.map(append_gold_answer).shuffle()
  val_dataset.to_csv(f"{mmlu_01}permuted_valset_{file_name}.csv")

  test_dataset = concatenate_datasets(test_tables).shuffle()
  create_datasets([test_dataset], [f"permuted_testset_{file_name}"], folder_path)
  return

folder_path = "/content/drive/MyDrive/685 Project/mmlu 01/"
create_permuted_set(permuted_train_table_names_8K, folder_path, "8k_new")
create_permuted_set(permuted_train_table_names_16K, folder_path, "16k_new")
create_permuted_set(permuted_train_table_names_32K, folder_path, "32k_new")



professional_psychology


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/612 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/69 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
table_name = 'electrical_engineering'

mmlu = load_dataset("cais/mmlu", table_name)["test"]
train_set = mmlu.select(range(100))
# train_dataset = concatenate_datasets(train_tables)
train_dataset = create_permuted_trainset(train_set)
train_dataset.to_csv('trainset.csv')

rest_set = mmlu.select(range(100, len(mmlu)))
split = rest_set.train_test_split(test_size=0.4)
val_set = split["train"]
test_set = split["test"]
val_dataset = val_set.map(preprocess_question, with_indices=True)
val_dataset = val_dataset.map(append_gold_answer).shuffle()
val_dataset.to_csv("valset.csv")

folder_path = ""
create_datasets([test_set], ["testset"], folder_path)

Generating test split:   0%|          | 0/145 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/16 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]